## Tutorial number 1
### Working with ethoscope data
The tutorial will guide you through loading data, visualisation and plotting using the common sleep functions

## 1. Load the dummy dataset

In [ ]:
import ethoscopy as etho
import pandas as pd

In [ ]:
# This tutorial requires version 1.1.7 or greater
etho.__version__

In [ ]:
# This creates the behavpy object, combining your data and metadata into a behavpy dataframe (remember to keep them in this order data, metadata - otherwise it will cause problems later down the line)
# The parameter check is used to see if the ids of the specimen in the data match those in the metadata. It also removes some columns that are not needed now you've downloaded the data
# Canvas specifies which plotting library you want to use. The choice is between plotly (default) and the matplotlib-frontend seaborn. 
# plotly allows to output interactive graphs but it's a bit more cumbersome to use and heavier in size. Seaborn / matplotlib is more of a standard, lighter and beautiful but only outputs static plots
# Try this notebook using both canvases to see the differences

df = pd.read_pickle('/home/lab/Desktop/habituation/pickles/habit_repnew_1_data.pkl')
pdf = pd.read_pickle('/home/lab/Desktop/habituation/pickles/habit_repnew_1_puff.pkl')
df.canvas

In [ ]:
fig, stats = pdf.plot_response_quantify(facet_col = 'exp', facet_arg=fargs, title = '')
fig.show()